In [1]:
load_packages = begin
	using Expokit, PROPACK, Arpack, SparseArrays
	using ProgressMeter, JLD, Plots
    ProgressMeter.ijulia_behavior(:clear)
	using Catalyst, JumpProcesses, StatsBase, DifferentialEquations
	using Interpolations
	using Revise
	local_mod = include("../src/DiscStochSim.jl")
	using .local_mod.DiscStochSim
end

┌ Warning: Replacing docs for `Main.DiscStochSim.FindLowestValuesPercent :: Union{Tuple{T}, Tuple{Vector{T}, Number}} where T` in module `Main.DiscStochSim`
└ @ Base.Docs docs/Docs.jl:243


In [2]:
rn = @reaction_network begin
    k₁, X --> 2X
    k₂, X + Y --> 2Y
    k₃, Y --> 0
end
model = DiscreteStochasticSystem(rn);

In [3]:
def_params = begin
	# reaction rates 
	rates = [1.0, 0.005, 0.6]
	# boundary 
	bounds = (0, 500) #(lower limit, upper limit)
	boundary_condition(x) = RectLatticeBoundaryCondition(x, bounds);
	# time interval and initial values
	δt = 0.005
	T = 0:δt:30
end

0.0:0.005:30.0

In [4]:
init_fsp_vars = begin
	global U₀ = CartesianIndex(50, 100)
	global 𝒮₀ = Set([U₀])
	global 𝒮₀ = expand!(𝒮₀, model, rates, 0.0, boundary_condition, 1);
	# initial probability vector (only for active states)
	global p₀ = zeros(𝒮₀ |> length)
	global p₀[FindElement(U₀, 𝒮₀)] = 1
end;

In [5]:
# initial probability vector (only for active states)
p₀ = zeros(X |> length)
p₀[DiscreteStochasticSimulation.FindElement(U₀, X)] = 1

global iter = 1
global pₜ = p₀
global sz = 0
global m =[]
# time intrerval 
sol = Array{SparseMatrixCSC,1}(undef, length(T))
global p = Progress(length(T))

for t ∈ T
    
    #expand space and assemble matrix
    global X, pₜ = Expand!(X, pₜ, model, rates , t, boundary_condition, 5)

    sz = length(X)
    A = MasterEquation(X, model, rates, boundary_condition, t)
     
    # solve system and normalize (using expokit (can also use numerical time steppers such as Runge-Kutta))
    global pₜ = expmv(δt, A, pₜ)
    
    # add add states in sparse arrays 
    I = [a[1] for a in X] 
    J = [a[2] for a in X] 
    sol[iter] = sparse(I, J, pₜ)
    
    # purge lowest 3% probability states (most will be zero; this is arbitrary as of now)
    X, pₜ = Purge!(X, pₜ, 3.0)
    push!(m, 1.0 - sum(pₜ))
    pₜ ./= sum(pₜ) # re-normalize the reduced state space
  
    # sparse matrix 
    iter+=1
    
    # comment following line if not using progress meter
    next!(p; showvalues = [
                           ("time (t)", t),
                           ("system size",sz),
                           (" ", " "), 
                           ("system size (after purge)",length(pₜ)),
                           ("num pruned states", sz-length(pₜ)),
                           ("pruned mass", m[end])])
end

LoadError: UndefVarError: `X` not defined in `Main`
Suggestion: check for spelling errors or missing imports.